# Introduction to Physics Simulation and URDFs


## Setting Up PyBullet in Colab

First, let’s set up our environment by installing PyBullet and importing the necessary modules. We’ll connect to PyBullet in DIRECT mode (no GUI). If you were running this on your own computer and wanted to see the simulation in a GUI window, you could use `p.connect(p.GUI)` instead, but that won’t work in Colab.

**Code**: Install and import PyBullet, and connect to the physics server.


In [1]:
!pip install -q pybullet

In [5]:
import sys
print(sys.executable)
print(sys.version)

/Users/henry/miniforge3/envs/pybullet_env/bin/python
3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:46:00) [Clang 18.1.8 ]


In [1]:
import PyQt5
%gui qt
import pybullet as p
import pybullet_data
import time

# Connect to PyBullet physics server in DIRECT mode (no GUI).
physicsClient = p.connect(p.DIRECT)
# Set search path to find URDFs (includes PyBullet's built-in urdf files).
p.setAdditionalSearchPath(pybullet_data.getDataPath())
print("PyBullet connected (ID {})".format(physicsClient))

PyBullet connected (ID 0)


pybullet build time: May 17 2025 21:05:07


In [6]:
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


Running the above should install PyBullet and print a confirmation that we’re connected. The physicsClient ID (usually 0) indicates a successful connection.

Output:
```
PyBullet connected (ID 0)
```

(Optional – for local desktop users only: If you are running this notebook on your own computer with PyBullet installed, you can use GUI mode to see the simulation. Uncomment the line below and comment out the DIRECT mode line to connect with GUI visualization. Remember, this won’t show a window in Colab.)


In [2]:
physicsClient = p.connect(p.GUI)  # Optional: use GUI mode if running locally

Version = 4.1 Metal - 89.4
Vendor = Apple
Renderer = Apple M4 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


## Loading a Simple Robot and Environment

Now that PyBullet is ready, let’s set up a basic simulation world:
1. **Add a ground plane**: Most simulations start with a floor. PyBullet has a ready-made plane URDF we can load.
2. **Define a simple robot URDF**: We will use a very basic two-link arm as our robot. To keep things self-contained, we’ll generate a URDF file for it on the fly. (In practice, you might load a URDF from disk. PyBullet also has example robots like KUKA arms, but those have many joints and details. Our simple arm will be easier to understand.)
3. **Load the robot URDF into the simulation**.


**Code**: Load a ground plane and create a simple two-link robot URDF file.

In [3]:
# physicsClient = p.connect(p.GUI)  
# Load a basic flat plane as ground.
plane_id = p.loadURDF("plane.urdf")
print("Loaded ground plane with ID:", plane_id)

# Define a simple two-link robot URDF as a string.
simple_urdf = """<?xml version="1.0"?>
<robot name="simple_two_link">
  <link name="base_link">
    <inertial>
      <mass value="1"/>
      <inertia ixx="1" iyy="1" izz="1" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <geometry><box size="0.1 0.1 0.1"/></geometry>
      <material name="blue"><color rgba="0 0 1 1"/></material>
    </visual>
  </link>
  <link name="link1">
    <inertial>
      <mass value="1"/>
      <inertia ixx="0.1" iyy="0.1" izz="0.1" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <origin xyz="0 0 0.5" rpy="0 0 0"/>
      <geometry><cylinder radius="0.05" length="1"/></geometry>
      <material name="green"><color rgba="0 1 0 1"/></material>
    </visual>
  </link>
  <link name="link2">
    <inertial>
      <mass value="1"/>
      <inertia ixx="0.1" iyy="0.1" izz="0.1" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <origin xyz="0 0 0.5" rpy="0 0 0"/>
      <geometry><cylinder radius="0.05" length="1"/></geometry>
      <material name="red"><color rgba="1 0 0 1"/></material>
    </visual>
  </link>
  <!-- Joint connecting base_link and link1 -->
  <joint name="joint1" type="revolute">
    <parent link="base_link"/>
    <child link="link1"/>
    <origin xyz="0 0 0.1" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit lower="-3.14" upper="3.14" effort="10" velocity="1"/>
  </joint>
  <!-- Joint connecting link1 and link2 -->
  <joint name="joint2" type="revolute">
    <parent link="link1"/>
    <child link="link2"/>
    <origin xyz="0 0 1" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit lower="-3.14" upper="3.14" effort="10" velocity="1"/>
  </joint>
</robot>"""
# Write the URDF string to a file.
with open("simple_arm.urdf", "w") as f:
    f.write(simple_urdf)
print("simple_arm.urdf file created.")


Loaded ground plane with ID: 0
simple_arm.urdf file created.


Let’s break down what we did in the URDF above:

- We created a robot named "simple_two_link".
- Defined three links: **base_link** (a small blue box, acting as the robot’s base), **link1** (a green cylinder of length 1 unit), and **link2** (a red cylinder of length 1 unit).
- Added two revolute (rotating) joints: **joint1** attaches base_link to link1 (with an axis of rotation along the Y-axis through the base, meaning link1 can swing up/down), and **joint2** attaches link1 to link2 (axis along link1’s Y-axis, so link2 can swing relative to link1). We gave each joint a range of motion from -3.14 to +3.14 radians (~±180 degrees).
- Note: We positioned link1’s joint origin at `<origin xyz="0 0 0.1"/>` relative to base_link (so link1 attaches at the top of the base box), and link2’s joint origin at `<origin xyz="0 0 1"/>` relative to link1 (so it attaches at link1’s end). All rotations are about the Y axis (`axis xyz="0 1 0"`), which for our orientation means a horizontal axis allowing the links to swing in a vertical plane (like an elbow bending).


Now that the URDF file is created, let’s load the robot into the simulation.

In [4]:
# Load the simple two-link robot URDF into PyBullet
robot_id = p.loadURDF("simple_arm.urdf", basePosition=[0,0,0], useFixedBase=True)
print("Loaded simple robot with ID:", robot_id)


Loaded simple robot with ID: 1



This is the result of simulating the robot on local.

<img src="https://i.postimg.cc/t4jzzmT3/urdf-model.png" alt="urdf_model" width="300"/>


We used `useFixedBase=True` so that the base_link is anchored at the given basePosition and will not fall due to gravity. The robot is now in the simulation. We also loaded a ground plane earlier (plane_id). By default, PyBullet has gravity enabled (set to Earth gravity 9.8 m/s² downward on the z-axis). If our robot’s base was not fixed, it would start falling. Since it’s fixed, it will stay in place – useful because this robot has no legs or supports.

The code prints the unique ID of the robot (an integer). We have our world set up: a ground and a simple robot.

Output:
```
Loaded ground plane with ID: 0
simple_arm.urdf file created.
Loaded simple robot with ID: 1
```

## Inspecting the Robot Model

Let’s explore some properties of the loaded robot to understand the structure:

**Code**: Query robot details (number of joints, joint names, etc).

In [5]:
num_joints = p.getNumJoints(robot_id)
print("Number of joints in the robot:", num_joints)

# Print the name and type of each joint
for ji in range(num_joints):
    info = p.getJointInfo(robot_id, ji)
    joint_name = info[1].decode('utf-8')
    joint_type = info[2]
    print(f" Joint {ji}: name='{joint_name}', type={joint_type}")


Number of joints in the robot: 2
 Joint 0: name='joint1', type=0
 Joint 1: name='joint2', type=0


When you run this, you should see that our robot has 2 joints (joint1 and joint2). PyBullet assigns each joint an index (0 and 1 in this case, since joints are numbered starting from 0). The joint type is given as an integer code – 0 typically means revolute (hinge) joint. The joint names come from our URDF.

Output:
```
Number of joints in the robot: 2
 Joint 0: name='joint1', type=0
 Joint 1: name='joint2', type=0
```

This confirms our robot’s two joints are loaded correctly. We can also query links. In PyBullet, the base link of a robot isn’t counted as a joint (the robot’s base is fixed or free in space). So with 2 joints, there are 3 links (base_link plus two others). We can verify the link names via the joint info as well (PyBullet’s getJointInfo returns child link names, etc.). Let’s also check the robot’s starting pose briefly.


In [6]:
# Get base position and orientation
base_pos, base_orn = p.getBasePositionAndOrientation(robot_id)
print("Robot base position (x,y,z):", base_pos)
print("Robot base orientation (as quaternion):", base_orn)

# Check initial joint states (angles)
for ji in range(num_joints):
    joint_state = p.getJointState(robot_id, ji)
    joint_angle = joint_state[0]  # position value
    print(f" Initial angle of joint {ji} ('{p.getJointInfo(robot_id, ji)[1].decode('utf-8')}') = {joint_angle:.2f} rad")


Robot base position (x,y,z): (0.0, 0.0, 0.0)
Robot base orientation (as quaternion): (0.0, 0.0, 0.0, 1.0)
 Initial angle of joint 0 ('joint1') = 0.00 rad
 Initial angle of joint 1 ('joint2') = 0.00 rad


By default, if we don’t specify anything, PyBullet loads the URDF with all joint angles at their default positions (usually 0). The base position should be (0,0,0) as we set, and orientation as a quaternion (0,0,0,1) meaning no rotation from world axes.

Output:
```
Robot base position (x,y,z): (0.0, 0.0, 0.0)
Robot base orientation (as quaternion): (0.0, 0.0, 0.0, 1.0)
 Initial angle of joint 0 ('joint1') = 0.00 rad
 Initial angle of joint 1 ('joint2') = 0.00 rad

```

The robot’s base is at the world origin, and the joints are initially at 0 radians (so link1 is unrotated, pointing presumably straight down from the base’s attachment point; link2 is straight in line with link1 since its joint is also 0).


## Simulating Physics
Even though we have a robot loaded, nothing has moved yet. The physics simulation advances in steps. In a real-time simulator, these steps would be advanced continuously. In PyBullet’s DIRECT mode, we manually step the simulation.

Right now, gravity is acting on the robot. But we fixed the base in place, and the arm links are connected by joints with no motors actively moving them. Because we didn’t apply any initial forces or velocities, the arm will just hang there in whatever pose we set (in this case, straight down). Let’s advance the simulation a bit to confirm nothing crazy happens (it shouldn’t).

**Code**: Step the simulation forward and see if anything changes.


In [7]:
# Run the simulation for a short time (e.g., 1 second of simulated time)
# PyBullet default time step is ~1/240 of a second, so 240 steps = 1 second.
for _ in range(240):
    p.stepSimulation()

# After simulation steps, check the joint angles again to see if they moved
for ji in range(num_joints):
    joint_angle = p.getJointState(robot_id, ji)[0]
    print(f"Post-simulation angle of joint {ji} = {joint_angle:.2f} rad")


Post-simulation angle of joint 0 = 0.00 rad
Post-simulation angle of joint 1 = 0.00 rad


: 

We stepped the simulation 240 times (which corresponds to roughly 1 second of simulation time at default settings). Since we didn’t apply any motor commands or external forces, and the base is fixed, the arm should remain in place. The joint angles printed after simulation should still be 0.00 rad for both joints, confirming nothing moved on its own.

Output:
```
Post-simulation angle of joint 0 = 0.00 rad
Post-simulation angle of joint 1 = 0.00 rad
```

Great! We have successfully:
- Set up a physics simulation environment,
- Loaded a robot described by a URDF,
- Stepped the simulation.
